In [ ]:
import json
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/capstone_data/train.json') as file:
    train_data = json.load(file)

with open('/content/drive/MyDrive/capstone_data/validation.json') as file:
    validation_data = json.load(file)

In [ ]:
def labelling(json_data):
    keywords, theme, recom = [], [], []
    theme_label = {}

    for news in json_data['berita']:
        for keyword in news['keywords']:
            keywords.append(keyword)
            theme.append(news['tema_berita'])
            recom.append(news['recommendation'])

        for tema in news['tema_berita']:
            if tema not in theme_label:
                theme_label[news['tema_berita']] = news['recommendation']

    return keywords, theme, recom, theme_label

In [ ]:
train_keywords, train_theme, train_recom, train_theme_label = labelling(train_data)

val_keywords, val_theme, val_recom, val_theme_label = labelling(validation_data)

In [ ]:
data = pd.DataFrame({
    'keywords': train_keywords,
    'theme': train_theme,
    'recommendation': train_recom
})

data.sample(10)

,keywords,theme,recommendation
57,polisi,polisi,"[Kejahatan, Polisi]"
122,rumah sakit,rumah sakit,"[Rumah Sakit, Penyakit, Kesehatan]"
8,bocor,kebakaran,"[Kebakaran, Pemadam Kebakaran]"
42,pertamina,kebakaran,"[Kebakaran, Pemadam Kebakaran]"
151,jantung,rumah sakit,"[Rumah Sakit, Penyakit, Kesehatan]"
100,keadilan,polisi,"[Kejahatan, Polisi]"
37,demo,kebakaran,"[Kebakaran, Pemadam Kebakaran]"
70,tindak pidana,polisi,"[Kejahatan, Polisi]"
87,keamanan,polisi,"[Kejahatan, Polisi]"
20,percikan,kebakaran,"[Kebakaran, Pemadam Kebakaran]"


In [ ]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(train_keywords)
train_sequences = tokenizer.texts_to_sequences(train_keywords)
word_index = tokenizer.word_index
max_sequence_length = max([len(seq) for seq in train_sequences])
train_padded_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length)

tokenizer.fit_on_texts(val_keywords)
val_sequences = tokenizer.texts_to_sequences(val_keywords)
word_index = tokenizer.word_index
max_sequence_length = max([len(seq) for seq in val_sequences])
val_padded_sequences = pad_sequences(val_sequences, maxlen=max_sequence_length)

In [ ]:
label_mapping = {'kebakaran': 0, 'polisi': 1, 'rumah sakit': 2}
num_classes = len(set(val_theme))

X_train = train_padded_sequences
y_train = tf.keras.utils.to_categorical([label_mapping[label] for label in train_theme], num_classes=num_classes)

X_test = val_padded_sequences
y_test = tf.keras.utils.to_categorical([label_mapping[label] for label in val_theme], num_classes=num_classes)

In [ ]:
y_test


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [ ]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, 100))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_train, y_train))

Epoch 1/100
11/11 [==============================] - 4s 80ms/step - loss: 1.1013 - accuracy: 0.2901 - val_loss: 1.0946 - val_accuracy: 0.3333
Epoch 2/100
11/11 [==============================] - 0s 13ms/step - loss: 1.0934 - accuracy: 0.4321 - val_loss: 1.0871 - val_accuracy: 0.6728
Epoch 3/100
11/11 [==============================] - 0s 16ms/step - loss: 1.0834 - accuracy: 0.8395 - val_loss: 1.0731 - val_accuracy: 0.9691
Epoch 4/100
11/11 [==============================] - 0s 14ms/step - loss: 1.0648 - accuracy: 0.9691 - val_loss: 1.0446 - val_accuracy: 0.9691
Epoch 5/100
11/11 [==============================] - 0s 12ms/step - loss: 1.0286 - accuracy: 0.9691 - val_loss: 0.9914 - val_accuracy: 0.9691
Epoch 6/100
11/11 [==============================] - 0s 13ms/step - loss: 0.9612 - accuracy: 0.9630 - val_loss: 0.8934 - val_accuracy: 0.9630
Epoch 7/100
11/11 [==============================] - 0s 12ms/step - loss: 0.8395 - accuracy: 0.9630 - val_loss: 0.7366 - val_accuracy: 0.9630
Epoch 

In [ ]:
test_sentence = """
rumah hangus terbakar

"""
test_sequence = tokenizer.texts_to_sequences([test_sentence])
padded_test_sequence = pad_sequences(test_sequence, maxlen=max_sequence_length)
prediction = model.predict(padded_test_sequence)[0]

recommended_hobby_index = tf.argmax(prediction).numpy()
recommended_hobby = [label for label, index in label_mapping.items() if index == recommended_hobby_index][0]

print("Rekomendasi: " + recommended_hobby)

1/1 [==============================] - 1s 701ms/step
Rekomendasi: kebakaran
